Steps:
- Scarping Job posting via Scrapy, Beautifulsoup or Selenium
- Visualize in Tableau or PowerBI

import necessary modules - first try with BeautifulSoup

In [71]:
import driver as driver
import pandas as pd
import re
import lxml
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn


Select wanted url

In [72]:
city = 'Zurich'
state = 'Zurich'
country = 'Switzerland'
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Zurich%2C%20Zurich%2C%20Switzerland&geoId=107814425&trk=public_jobs_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0'
no_of_jobs = 50




open windows with url stated above in

In [73]:
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

to show more jobs than the initial 25

In [74]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(20)



parsing the visible webpage

In [75]:
pageSource = driver.page_source
soup = BeautifulSoup(pageSource,  "html.parser")



Parsing content

In [76]:
first_job = soup.find('h3', class_ = 'result-card__title job-result-card__title').text
print(first_job)

first_company = soup.find('a', class_ = 'result-card__subtitle-link job-result-card__subtitle-link').text
print(first_company)

Data Analyst
EPAM Systems


Create job container

In [77]:
job_container = soup.find('ul', class_="jobs-search__results-list")
print(len(job_container))

48


create for loop to look for all jobs

In [78]:
print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 48 jobs.


In [79]:
job_titles = []
companies = []
locations = []
time = []
date = []


for jobs in job_container:
    post_title = jobs.find('h3', class_ = 'result-card__title job-result-card__title').text
    job_titles.append(post_title)

    post_company = jobs.find('h4', class_ = 'result-card__subtitle job-result-card__subtitle').text
    companies.append(post_company)

    post_locations = jobs.find('span', class_ = 'job-result-card__location').text
    locations.append(post_locations)

    post_date = jobs.select_one('time')['datetime']
    date.append(post_date)


In [80]:
ml = {'Title': job_titles, 'Company': companies, 'Locations': locations, 'Post Date': date}
print(ml)

{'Title': ['Data Analyst', 'Data Analyst (all genders, 80-100%)', 'Global Digital Data Analyst', 'Digital Analyst (w/m/d)', 'Junior Business Analyst', 'Business Intelligence Analyst (80%)', 'Data Consultant', 'Reporting/ Data Analyst (Banking - Operations)', 'Junior Business Analyst ', 'Business Analyst', 'Data Analyst (temporary)', 'Business Analyst', 'Data Scientist - QuantumBlack', 'Business Analyst', 'Business Analyst', 'Product Analyst, Calendar and Tasks', 'Controller / Data Analyst / Reporting Specialist', 'Data Engineer', 'Business Analyst for Data Management', 'Business Analyst', 'Business Analyst 100% (m/w)', 'Data Engineer (80 - 100%)', 'Business Analyst – Data Quality #164527', 'Data Science Developer', 'Business Analyst/in', 'Data Engineer', 'Analytics Engineer', 'Website Business Analyst and Product manager (100%)', 'Data Engineer', 'Big Data Analyst/Developer', 'Business Insights Analyst, Direct-to-Consumer - Global', 'HR Data Administrator & Analyst (50% - 60%)', 'Data 

In [81]:
df = pd.DataFrame(ml)
df.head(1)

,Title,Company,Locations,Post Date
0,Data Analyst,EPAM Systems,"Zurich, Zurich, Switzerland",2021-01-15


Separate City from State and Country


In [82]:
new = df.Locations.str.split(',', n = 0, expand = True)

In [83]:
new2 = new.rename(columns = {0: 'City2'})

In [84]:
new2


,City2,1,2
0,Zurich,Zurich,Switzerland
1,Seon,Aargau,Switzerland
2,Zurich,Switzerland,None
3,Schlatt (Zürich),Zurich,Switzerland
4,Zurich,Zurich,Switzerland
5,Zurich,Zurich,Switzerland
6,Schlieren,Zurich,Switzerland
7,Zurich,Zurich,Switzerland
8,Zurich,Zurich,Switzerland
9,Zurich,Switzerland,None


merge df and new, drop Locations as it is redundant now

In [85]:
df2 = df.join(new2, how = 'left')
df2.drop(columns = ['Locations',1 , 2], inplace = True)
df2.head(4)

,Title,Company,Post Date,City2
0,Data Analyst,EPAM Systems,2021-01-15,Zurich
1,"Data Analyst (all genders, 80-100%)",MAMMUT Sports Group AG,2021-01-21,Seon
2,Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
3,Digital Analyst (w/m/d),Migros-Genossenschafts-Bund,2021-01-19,Schlatt (Zürich)


In [86]:
new3 = df2.City2.str.split(' ', expand=True)
new4 = new3.rename(columns = {0: 'City'})
new4.head(4)

,City,1,2
0,Zurich,None,None
1,Seon,None,None
2,Zurich,None,None
3,Schlatt,(Zürich),None


In [87]:
df4 = df2.join(new4, how = 'left')
df4.drop(columns = [1, 2, "City2"], inplace = True)
df4.head(4)



,Title,Company,Post Date,City
0,Data Analyst,EPAM Systems,2021-01-15,Zurich
1,"Data Analyst (all genders, 80-100%)",MAMMUT Sports Group AG,2021-01-21,Seon
2,Global Digital Data Analyst,La Prairie Group AG,2021-01-17,Zurich
3,Digital Analyst (w/m/d),Migros-Genossenschafts-Bund,2021-01-19,Schlatt


create CSV

In [88]:
df4.to_csv('C:/Users/Marc\Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/jobs_zurich_50.csv')
